# Week 3 — Part 02: Prompts as API contracts

**Estimated time:** 60–90 minutes

## What success looks like (end of Part 02)

- You can write a prompt that acts like a contract: explicit keys, explicit constraints, and clear handling for missing data.
- You can validate a model’s output deterministically (parse + schema checks).

### Checkpoint

- You can run `parse_contract_output(...)` on at least 2 simulated outputs.
- Your prompt explicitly bans extra keys and non-JSON text.

## Learning Objectives

- Treat prompts as specifications (contracts)
- Define clear preconditions and postconditions
- Design strict JSON output contracts
- Create validators that make outputs checkable

## Overview

A strong prompt is not “clever wording”. It’s a **specification**.

If you treat the model like a service, your prompt is the API contract.

We’ll practice writing prompts with:

- explicit **role** and **task**
- explicit **output schema**
- explicit **constraints** (no extra keys, no markdown)
- explicit **refusal conditions** (what to do if info is missing)

And we’ll build a small validator so outputs are checkable.

In [ ]:
import json
from dataclasses import dataclass
from typing import Optional, Set


@dataclass(frozen=True)
class Extracted:
    person: Optional[str]
    company: Optional[str]


def validate_exact_keys(obj: dict, required_keys: Set[str]) -> None:
    extra = set(obj.keys()) - required_keys
    missing = required_keys - set(obj.keys())
    if missing:
        raise ValueError(f"missing keys: {sorted(missing)}")
    if extra:
        raise ValueError(f"extra keys: {sorted(extra)}")


def parse_contract_output(text: str) -> Extracted:
    data = json.loads(text)
    if not isinstance(data, dict):
        raise ValueError("output must be a JSON object")
    validate_exact_keys(data, {"person", "company"})

    person = data.get("person")
    company = data.get("company")

    if person is not None and not isinstance(person, str):
        raise ValueError("person must be string or null")
    if company is not None and not isinstance(company, str):
        raise ValueError("company must be string or null")

    return Extracted(person=person, company=company)


print(parse_contract_output('{"person": "Ada", "company": null}'))

## Exercise: Write a contract prompt

Below is a contract prompt template. Your job is to adjust it for new tasks while keeping it checkable.

Key properties:

- “Return ONLY valid JSON”
- “exactly these keys”
- “Use null if not found”

In [ ]:
def build_extraction_prompt(text: str) -> str:
    # Non-verbatim: same idea, slightly different wording and ordering.
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Output: ONLY JSON with keys person, company.\n"
        "Rules: no markdown, no additional keys, use null when missing.\n\n"
        f"Input text:\n{text}\n"
    )


print(build_extraction_prompt("Ada Lovelace founded nothing."))

In [ ]:
def simulate_model_output(prompt: str) -> str:
    # Simulated outputs for practice (stand-in for real LLM call)
    if "Ada" in prompt:
        return '{"person": "Ada Lovelace", "company": null}'
    return '{"person": null, "company": null}'


raw = simulate_model_output(build_extraction_prompt("Ada Lovelace wrote notes."))
print("raw:", raw)
print("parsed:", parse_contract_output(raw))

In [ ]:
def build_refusal_contract_todo(text: str) -> str:
    """TODO: extend the contract to return either:

    - {"ok": true, "person": ..., "company": ...}
    - OR {"ok": false, "error": "..."}

    Keep it strict:

    - return ONLY JSON
    - no additional keys
    - use null for missing fields when ok=true
    """
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Output: ONLY JSON with exactly these keys: ok, person, company, error.\n"
        "Rules: no markdown, no additional keys.\n"
        "Rules: if extraction is possible, set ok=true, set error=null.\n"
        "Rules: if extraction is not possible, set ok=false, set person=null, company=null, and set error to a short reason.\n\n"
        f"Input text:\n{text}\n"
    )


print(build_refusal_contract_todo("Ada Lovelace wrote notes."))

## Self-check

- Does your prompt define exact keys?
- Does it forbid extra text?
- Does it define what to do when info is missing?

## References

- Prompting guide: https://www.promptingguide.ai/
- Anthropic cookbook: https://github.com/anthropics/anthropic-cookbook

## Appendix: Solutions (peek only after trying)

Reference implementation for `build_refusal_contract_todo`.

In [ ]:
def build_refusal_contract_todo(text: str) -> str:
    return (
        "Role: Information extraction engine.\n"
        "Task: Extract a person name and a company name.\n"
        "Return ONLY valid JSON.\n"
        "Output schema (exact keys): ok, person, company, error\n"
        "Constraints: no markdown, no prose, no extra keys.\n"
        "If a value is missing, use null.\n"
        "If you cannot comply, return ok=false and a short error message.\n\n"
        f"INPUT:\n{text}\n"
    )


print(build_refusal_contract_todo("No names here."))